In [3]:
import pandas as PD
import geopandas as GPD
import numpy as NP
import matplotlib.pyplot as PLT
from tqdm import tqdm as TQDM
from scipy.spatial.distance import cdist
from scipy.optimize import least_squares

In [45]:
gdf = GPD.read_file('./data/muese.shp')
gdf = gdf.reset_index().rename(columns={"index":"i"})
gdf['i'] = gdf['i'].astype(str)
gdf.head()

,i,cadmium,copper,lead,zinc,elev,dist,om,ffreq,soil,lime,landuse,dist.m,x,y,geometry
0,0,11.7,85.0,299.0,1022.0,7.909,0.001358,13.6,1,1,1,Ah,50.0,181072.0,333611.0,POINT (5.758536241675762 50.99156215667816)
1,1,8.6,81.0,277.0,1141.0,6.983,0.012224,14.0,1,1,1,Ah,30.0,181025.0,333558.0,POINT (5.757863018200728 50.99108790288359)
2,2,6.5,68.0,199.0,640.0,7.800,0.103029,13.0,1,1,1,Ah,150.0,181165.0,333537.0,POINT (5.759855417830628 50.99089274333686)
3,3,2.6,81.0,116.0,257.0,7.655,0.190094,8.0,1,2,0,Ga,270.0,181298.0,333484.0,POINT (5.761745770023087 50.99041023559325)
4,4,2.8,48.0,117.0,269.0,7.480,0.277090,8.7,1,2,0,Ah,380.0,181307.0,333330.0,POINT (5.761862707698853 50.98902557091748)


In [46]:
x = gdf[['x', 'y']].values

In [47]:
x.shape

(155, 2)

In [48]:
y = gdf['lead'].values
y = y[:, NP.newaxis]

In [49]:
v = NP.triu(cdist(y, y, metric='sqeuclidean'))
v.shape

(155, 155)

In [61]:
df = PD.DataFrame(v, columns=[str(i) for i in range(v.shape[0])]).reset_index().rename(columns={'index':'i'})
df['i'] = df['i'].astype(str)
dfm = PD.melt(df, id_vars='i', value_vars=[str(i) for i in range(v.shape[0])], value_name='v', var_name='j').query('v != 0').reset_index(drop=True)
dfm.dtypes

i     object
j     object
v    float64
dtype: object

In [62]:
from shapely.geometry import Point, LineString
dfm = PD.merge(dfm, gdf[['i', 'geometry']], on='i').rename(columns={'geometry':'geometry_i'})
dfm = PD.merge(dfm, gdf[['i', 'geometry']].rename(columns={'i':'j'}), on='j' ).rename(columns={'geometry':'geometry_j'})
dfm['geometry'] = dfm[['geometry_i', 'geometry_j']].apply(lambda x: LineString(x), axis=1)

In [65]:
dfm = GPD.GeoDataFrame(dfm[['i', 'j', 'v', 'geometry']], geometry='geometry')
dfm.to_file('./data/muese_variance.shp', driver='ESRI Shapefile', encoding = 'utf-8')

CPLE_NotSupported in driver ESRI Shapefile does not support creation option ENCODING
